In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
df = pd.read_csv("Housing_FOR_LR.csv")

In [39]:
df.head()

,Unnamed: 0,Lot Frontage,Lot Area,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,...,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial
0,0,141.0,31770,6,5,1960,1960,112.0,639.0,0.0,...,0,0,0,0,1,0,0,0,1,0
1,1,80.0,11622,5,6,1961,1961,0.0,468.0,144.0,...,0,0,0,0,1,0,0,0,1,0
2,2,81.0,14267,6,6,1958,1958,108.0,923.0,0.0,...,0,0,0,0,1,0,0,0,1,0
3,3,93.0,11160,7,5,1968,1968,0.0,1065.0,0.0,...,0,0,0,0,1,0,0,0,1,0
4,4,74.0,13830,5,5,1997,1998,0.0,791.0,0.0,...,0,0,0,0,1,0,0,0,1,0


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2925 entries, 0 to 2924
Columns: 275 entries, Unnamed: 0 to Sale Condition_Partial
dtypes: float64(11), int64(264)
memory usage: 6.1 MB


### Set independent variables(X)  and dependent variables (Y)

In [41]:
X = df.drop('SalePrice',axis=1)
y = df['SalePrice']

### Split up X and y into a training set and test set

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=101)

### Scale the X features, becouse the dataset features has a variety of scales and units. 

In [44]:
from sklearn.preprocessing import StandardScaler

In [45]:
scaler = StandardScaler()

In [46]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

### ElasticNet model with scikit-learn

In [47]:
from sklearn.linear_model import ElasticNet

In [48]:
base_elastic_model = ElasticNet()

### Create a dictionary parameter grid of values 

In [49]:
param_grid = {'alpha':[0.1,1,5,10,50,100],
              'l1_ratio':[.1, .5, .7, .9, .95, .99, 1]}

### Create a GridSearchCV object and run a grid search for optimalization the model 

In [50]:
from sklearn.model_selection import GridSearchCV

In [51]:
grid_model = GridSearchCV(estimator=base_elastic_model,
                          param_grid=param_grid,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          verbose=1)

In [52]:
grid_model.fit(scaled_X_train,y_train)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139101145880.1053, tolerance: 1355206692.5276787
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 165797199240.36267, tolerance: 1307913805.6588454
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 133027479663.31421, tolerance: 1415056940.006106
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 212311548490.46887, tolerance: 1438198040.088288
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4854830882.525513, tolerance: 1345680018.2551236
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 13272388015.997314, tolerance: 1355206692.5276787
  model = cd_fast.enet_coordinate_descent(
C:\Users\Patrycja\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: C

GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 1, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [53]:
grid_model.best_params_

{'alpha': 100, 'l1_ratio': 1}

### Evaluate model on unseen data

In [57]:
y_pred = grid_model.predict(scaled_X_test)

In [58]:
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score

In [59]:
mean_absolute_error(y_test,y_pred)

14195.354916674858

In [60]:
np.sqrt(mean_squared_error(y_test,y_pred))

20558.508635060218

In [61]:
r2_score(y_test,y_pred)

0.918971534572871

In [62]:
np.mean(df['SalePrice'])

180815.53743589742